Installing required packages/libraries

In [281]:
#!pip install nltk

In [282]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import warnings
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_columns',None)
# Import relevant packages

import nltk

import re 

import string
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords  # Remove useless words
from nltk.stem.lancaster import LancasterStemmer  # Convert words to base form; aggressive
nltk.download('punkt')
# Import packages that help us to create document-term matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [283]:
df1=pd.read_csv('ratings_prediction(dataset).csv')
df1

,Unnamed: 0,Rating,Review_head,Full_review
0,0,5,Super!,super bbb prodact amazing bass effects speake...
1,1,5,Great product,supper sound lacky
2,2,5,Must buy!,nice sound system . superb bass audio . aweso...
3,3,4,Worth the money,nice
4,4,4,Nice product,"nice product , good sound"
...,...,...,...,...
109551,8887,5,Wonderful,Beautiful coronavirus
109552,8888,5,Must buy!,Laptop hot
109553,8889,1,Utterly Disappointed,Within 12 days of purchasing it is out of orde...
109554,8890,3,Just okay,It is too small and handy. Able to use as Lapt...


The dataset has 45,598 rows and 4 columns(features):
    1)Unnamed:0  -It contains the previous indices of the dataframes that were concatenated into this one.
    2)Rating(dependent)  -It contains the ratings associated with the reviews.
    3)Review_head  -It contains the headings associated with the reviews respectively.
    4)Full_review  -It contains the full review text.
    
    As apparent the Unnamed:0 feature has no significance in working towards our goal of model building which can predict the rating by seeing the review.
        

In [284]:
df2=pd.read_csv('rating_prediction(extended).csv')
df2

,Unnamed: 0,Rating,Review_head,Full_review
0,0,2,Could be way better,Price is high.
1,1,5,Brilliant,Superb quality nice panels good working safe l...
2,2,5,Simply awesome,Fine product
3,3,4,Worth the money,Almost good product
4,4,5,Wonderful,Nice product...
...,...,...,...,...
38175,115687,4,Really Nice,small..but good for beginners
38176,115688,5,Highly recommended,Super for proficition photo grapher
38177,115689,3,Just okay,in the description it was mentioned as as alum...
38178,115691,5,Terrific purchase,this is the best stepilizer for low angle shot...


The dataset(df2) was extracted/scraped later due to the lack of sufficient data.Hence, it was concatenated with the former dataframe(df1) later in the worksheet.

In [285]:
df2.drop('Unnamed: 0',axis=1,inplace=True)

In [286]:
df2['Rating'].value_counts()

5    19863
4     6689
1     6098
3     3496
2     2026
-        8
Name: Rating, dtype: int64

In [287]:
np.where(df2['Rating']=='-')

(array([ 5316,  5317, 21807, 21808, 21809, 21810, 21811, 21812],
       dtype=int64),)

In [288]:
drop_list=[5316,5317,21807,21808,21809,21810,21811,21812]
df2.drop(drop_list,inplace=True)
df2['Rating'].value_counts()

5    19863
4     6689
1     6098
3     3496
2     2026
Name: Rating, dtype: int64

In [289]:
#dropping the irrelevant feature
df1.drop('Unnamed: 0',axis=1,inplace=True)

In [290]:
df1['Rating'].value_counts()

5    61525
1    19278
4    17648
3     7499
2     3606
Name: Rating, dtype: int64

The target feature is largely imbalanced with varying amounts of samples for different categories.
Splitting the dataset into different dataframes as per the category they fall into,with a goal of balancing the dataset.

In [291]:
df1.drop_duplicates(keep='first',inplace=True)

In [293]:
df1['Rating'].value_counts()

5    25495
1     8934
4     8110
3     3609
2     1989
Name: Rating, dtype: int64

In [294]:
df1.isnull().sum()

Rating         0
Review_head    0
Full_review    0
dtype: int64

Since there were a lot of words with similar implications were used eg: run,running etc ,Lanchester Stemmmer was used to stem the words to their core.

In [295]:
#Lancaster stemmer
lnc=LancasterStemmer()

In [296]:
df2.isnull().sum()

Rating         0
Review_head    0
Full_review    0
dtype: int64

In [297]:
#defining a function to remove stopwords from the text
def clean(text):
    try:
        cleaned=''
        stop_words=stopwords.words('english')
        lower_text=text.lower() #converting the text data to lowercase because the stopwords are in lowercase
        for sent_token in sent_tokenize(lower_text): #Splitting the text into tokens of sentences
            for word in nltk.word_tokenize(sent_token):#splitting the tokens of sentences into tokens of words
                if word not in stop_words:
                    stemmed=lnc.stem(word)
                    cleaned=cleaned+' '+stemmed
            #cleaned=cleaned+' '   So that the words dont merge when the '.' is removed as a punctuation
    except Exception as e:
        print("exception in",text,e )
    return cleaned

In [298]:
df2['Rating']=df2['Rating'].astype('int64')

In [299]:
df=pd.concat([df1,df2],ignore_index=True)

In [302]:
#Removing stopwords from the text data

for i in range(len(df)):
    df['Full_review'][i]=clean(df['Full_review'][i])
    

In [303]:
df['Rating'].value_counts()

5    45358
1    15032
4    14799
3     7105
2     4015
Name: Rating, dtype: int64

In [304]:
cat_1=df[df['Rating']==1]
cat_1

,Rating,Review_head,Full_review
17,1,Did not meet expectations,poor bas
32,1,Very poor,bluetoo bad
41,1,Worthless,poor qual
45,1,Useless product,purchas produc
48,1,Worthless,nyc sound bass smal ok pric
...,...,...,...
86262,1,Not recommended at all,univers stabl plast alumin
86264,1,Worthless,n't wast money ! ! !
86270,1,Don't waste your money,low qual
86284,1,Did not meet expectations,plast produc


In [305]:
cat_2=df[df['Rating']==2]
cat_3=df[df['Rating']==3]
cat_4=df[df['Rating']==4]
cat_5=df[df['Rating']==5]

Proceed with data analysis of the text data with respect to the rating.Web scrap more for your test dataset.

In [306]:
df[df['Rating']==2]

,Rating,Review_head,Full_review
38,2,Could be way better,sound qual good
42,2,Nice,bass real poor overal good produc
58,2,Could be way better,"guaranty 3 month , 'm disappoint ic bad two t..."
100,2,Not good,ok
115,2,Moderate,bhag produc
...,...,...,...
86255,2,Could be way better,writ
86258,2,Bad quality,"expect much worst qual , poor mat qual ,"
86278,2,Expected a better product,good handl shak much
86288,2,Could be way better,produc seem lightweight pro con . screw botto...


In [307]:
df['Full_review'][45000]

' port , perfect synt nee .'

In [308]:
alpha=lambda x:re.sub(r"\[[0-9]*\]",' ',x)
    

In [309]:
alpha2=lambda x:re.sub(r'\s+',' ',x)

In [310]:
alpha3=lambda x:x.lower()

In [311]:
alpha4=lambda x:re.sub(r'\d',' ',x)

In [312]:
alpha5=lambda x:re.sub(r'\s+'," ",x)

In [313]:
alpha6=lambda x:x.translate(str.maketrans("","",string.punctuation))

In [314]:
#converting the datatypes from string to object
list1=[cat_1,cat_2,cat_3,cat_4,cat_5]
for i in list1:
    i['Full_review']=i['Full_review'].astype(str)
    i['Review_head']=i['Review_head'].astype(str)

In [315]:
for i in list1:
    i['Full_review']=i['Full_review'].map(alpha).map(alpha2).map(alpha3).map(alpha4).map(alpha5).map(alpha6)

In [379]:
for i in list1:
    i['Review_head']=i['Review_head'].map(alpha).map(alpha2).map(alpha3).map(alpha4).map(alpha5).map(alpha6)

In [380]:
df['Rating'].value_counts()

5    45358
1    15032
4    14799
3     7105
2     4015
Name: Rating, dtype: int64

In [381]:
#"didn't" in stop_words

In [382]:
c_vect1=CountVectorizer()
c_vect1.fit_transform(cat_1['Full_review'])


<15032x5833 sparse matrix of type '<class 'numpy.int64'>'
	with 113977 stored elements in Compressed Sparse Row format>

In [383]:
#print(c_vect.get_feature_names())

In [384]:
#The words used can be checked out along with their frequency
check=c_vect1.vocabulary_

In [385]:
def sort_dict_by_value(d, reverse = False):
    
      return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))


In [386]:

print("\nSort (descending) the said dictionary elements by value:")
print(sort_dict_by_value(check, True))


Sort (descending) the said dictionary elements by value:
{'𝙞𝙣': 5832, '𝘽𝙖𝙙': 5831, 'zyad': 5830, 'zoom': 5829, 'zook': 5828, 'zomby': 5827, 'zirakp': 5826, 'zip': 5825, 'zeroo': 5824, 'zero': 5823, 'zenphon': 5822, 'zenfon': 5821, 'zee': 5820, 'zebron': 5819, 'ytmus': 5818, 'yt': 5817, 'yrs': 5816, 'yr': 5815, 'yoyr': 5814, 'youtubers': 5813, 'youtubego': 5812, 'youtube': 5811, 'youtub': 5810, 'yousless': 5809, 'yourcustomer': 5808, 'your': 5807, 'youits': 5806, 'you': 5805, 'yono': 5804, 'yonguo': 5803, 'yog': 5802, 'yo': 5801, 'yl': 5800, 'yfit': 5799, 'yeti': 5798, 'yet': 5797, 'yesterday': 5796, 'yess': 5795, 'yellow': 5794, 'yeh': 5793, 'years': 5792, 'yearold': 5791, 'yearit': 5790, 'year': 5789, 'yeah': 5788, 'ye': 5787, 'yar': 5786, 'yantralay': 5785, 'ya': 5784, 'xtend': 5783, 'xt': 5782, 'xs': 5781, 'xr': 5780, 'xper': 5779, 'xiom': 5778, 'xioam': 5777, 'xiaom': 5776, 'xiamo': 5775, 'xforc': 5774, 'xerox': 5773, 'xcpt': 5772, 'xaom': 5771, 'wwwsonycoinproductkdl': 5770, 'wvg

In [387]:
df_new=pd.concat([cat_1[:4000],cat_2[:4000],cat_3[:4000],cat_4[:4000],cat_5[:4000]])
df_new

,Rating,Review_head,Full_review
17,1,did not meet expectations,poor bas
32,1,very poor,bluetoo bad
41,1,worthless,poor qual
45,1,useless product,purchas produc
48,1,worthless,nyc sound bass smal ok pric
...,...,...,...
7649,5,wonderful,model
7650,5,wonderful,sup produc ❤️
7655,5,terrific,😍this nic produc lov n s work prop n smoo mus...
7656,5,highly recommended,superb also look awesom charg ind nt show mac...


In [398]:
#df_new.drop_duplicates(keep='first',inplace=False)

The tfidf vectorizer was also tried but the count vectorizer yeilded better results.

In [400]:

#tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
#col_list=['Full_review','Review_head']
#for i in col_list:
 #   x= tfv.fit_transform(df_new[i])
#y=df_new['Rating']

CountVectorizer

In [409]:
c_vect=CountVectorizer(ngram_range=(1,1))
col_list=['Full_review','Review_head']
for i in col_list:
    x=c_vect.fit_transform(df_new[i])
y=df_new['Rating']

In [410]:
y.value_counts()

5    4000
4    4000
3    4000
2    4000
1    4000
Name: Rating, dtype: int64

In [411]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression

LOGISTIC REGRESSION

In [413]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=42)
lr1=LogisticRegression()
#tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
#x_train_tfv1=tfv.fit_transform(x_train)
#x_test_tfv1=tfv.transform(x_test)
lr1.fit(x_train,y_train)
pred_log=lr1.predict(x_test)
print(accuracy_score(y_test,pred_log))
print(classification_report(y_test,pred_log))

0.9512
              precision    recall  f1-score   support

           1       0.95      0.98      0.97       991
           2       0.97      0.97      0.97      1004
           3       0.96      0.94      0.95      1010
           4       0.93      0.92      0.93      1030
           5       0.94      0.95      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



SUPPORT VECTOR MACHINE(LinearSVC)

In [414]:
from sklearn.svm import LinearSVC
svm=LinearSVC(class_weight='balanced')
svm.fit(x_train,y_train)
pred_svm=svm.predict(x_test)
print(accuracy_score(y_test,pred_svm))
print(classification_report(y_test,pred_svm))

0.9516
              precision    recall  f1-score   support

           1       0.95      0.98      0.97       991
           2       0.97      0.98      0.97      1004
           3       0.96      0.94      0.95      1010
           4       0.93      0.92      0.92      1030
           5       0.94      0.95      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



KNEIGHBORS CLASSIFIER

In [415]:
from sklearn.neighbors import KNeighborsClassifier
knc=KNeighborsClassifier(n_neighbors=3)
knc.fit(x_train,y_train)
pred_knc=knc.predict(x_test)
print(accuracy_score(y_test,pred_knc))
print(classification_report(y_test,pred_knc))

0.9476
              precision    recall  f1-score   support

           1       0.93      0.98      0.95       991
           2       0.97      0.98      0.97      1004
           3       0.97      0.93      0.95      1010
           4       0.93      0.91      0.92      1030
           5       0.94      0.95      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



DECISION TREE CLASSIFIER

In [417]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion='gini')
dtc.fit(x_train,y_train)
pred_dtc=dtc.predict(x_test)
print(accuracy_score(y_test,pred_dtc))
print(classification_report(y_test,pred_dtc))

0.9516
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       991
           2       0.98      0.98      0.98      1004
           3       0.96      0.94      0.95      1010
           4       0.93      0.92      0.92      1030
           5       0.94      0.94      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



RANDOM FOREST CLASSIFIER

In [427]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
pred_rfc=rfc.predict(x_test)
print(accuracy_score(y_test,pred_rfc))
print(classification_report(y_test,pred_rfc))

0.9516
              precision    recall  f1-score   support

           1       0.95      0.98      0.97       991
           2       0.98      0.98      0.98      1004
           3       0.96      0.94      0.95      1010
           4       0.94      0.92      0.93      1030
           5       0.94      0.94      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



CROSS-VALIDATION

In [421]:
from sklearn.model_selection import cross_val_score

In [422]:
#Logistic Regression
cv=cross_val_score(lr1,x,y,cv=10)
print("mean score:",cv.mean())
print("cross val score:",cv)

mean score: 0.9481999999999999
cross val score: [0.9625 0.966  0.957  0.967  0.915  0.929  0.944  0.9715 0.927  0.943 ]


In [423]:
#KNeighbors Classifier
cv=cross_val_score(knc,x,y,cv=10)
print("mean score:",cv.mean())
print("cross val score:",cv)

mean score: 0.45555
cross val score: [0.452  0.4275 0.4525 0.4615 0.427  0.486  0.4855 0.436  0.424  0.5035]


In [424]:
#Linear SVC
cv=cross_val_score(svm,x,y,cv=10)
print("mean score:",cv.mean())
print("cross val score:",cv)

mean score: 0.9490000000000001
cross val score: [0.963  0.9665 0.956  0.968  0.9195 0.9295 0.946  0.971  0.926  0.9445]


In [425]:
#Decision Tree Classifier
cv=cross_val_score(dtc,x,y,cv=10)
print("mean score:",cv.mean())
print("cross val score:",cv)

mean score: 0.94865
cross val score: [0.9645 0.966  0.9555 0.9655 0.9185 0.925  0.9475 0.974  0.9255 0.9445]


In [428]:
#Random Forest Classifier
cv=cross_val_score(rfc,x,y,cv=10)
print("mean score:",cv.mean())
print("cross val score:",cv)

mean score: 0.94985
cross val score: [0.964  0.969  0.9565 0.965  0.92   0.9275 0.9475 0.975  0.929  0.945 ]


All the algorithms performed well except the KNeighbors Classifier

Proceeding to Roc-Auc scoring

In [455]:
from sklearn.metrics import roc_curve,roc_auc_score

In [462]:
#RandomForest
rr=rfc.predict_proba(x_test)
print('Random Forest test roc-auc: {}'.format(roc_auc_score(y_test,rr,multi_class='ovo')))

Random Forest test roc-auc: 0.9854065308963614


In [463]:
#Logistic Regression
lr=lr1.predict_proba(x_test)
print('Logistic Regression test roc-auc: {}'.format(roc_auc_score(y_test,lr,multi_class='ovo')))

Logistic Regression test roc-auc: 0.9829373888930025


In [465]:
#Decision Tree Classifier
dt=dtc.predict_proba(x_test)
print('Decision Tree Classifier test roc-auc: {}'.format(roc_auc_score(y_test,dt,multi_class='ovo')))

Decision Tree Classifier test roc-auc: 0.9714124975375761


In [466]:
knn=knc.predict_proba(x_test)
print('KNeighbors Classifier test roc-auc: {}'.format(roc_auc_score(y_test,knn,multi_class='ovo')))

KNeighbors Classifier test roc-auc: 0.9778167952660128


The Random Forest Classifier has been selected as the best model as it has the lowest difference between the accuracy_score and the cross_val_score,a very good f-1 score and the highest roc_auc_score.
The f-1 score being the most important metric in reaching this conclusion.
Hyperparameter Tuning the Random Forest Classifier.
The LinearSVC algorithm cannot be used here as it does not have the predict_proba method.

In [475]:
from sklearn.model_selection import GridSearchCV
model=RandomForestClassifier()
parameters={'criterion':['gini','entropy'],'n_estimators':[100,200,300,500],'max_depth':[None,2,3,5]}
grid=GridSearchCV(estimator=model,param_grid=parameters,cv=5)
print(grid)
grid.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3, 5],
                         'n_estimators': [100, 200, 300, 500]})


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3, 5],
                         'n_estimators': [100, 200, 300, 500]})

In [480]:
print(grid.best_score_)
print(grid.best_estimator_.n_estimators)
print(grid.best_params_)


0.9485333333333333
500
{'criterion': 'entropy', 'max_depth': None, 'n_estimators': 500}


In [481]:
#Tuning the parameters with the best estimators
rfc_final=RandomForestClassifier(criterion='entropy',max_depth=None,n_estimators=500)
rfc_final.fit(x_train,y_train)
pred_rfc_final=rfc_final.predict(x_test)
print(accuracy_score(y_test,pred_rfc_final))
print(classification_report(y_test,pred_rfc_final))

0.9514
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       991
           2       0.98      0.98      0.98      1004
           3       0.95      0.94      0.95      1010
           4       0.94      0.92      0.93      1030
           5       0.94      0.94      0.94       965

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



Saving the tuned Random Forest model along with the Count Vectorizer

In [485]:
import pickle
filename2='Rating_prediction(vectorizer).sav'
filename1='Rating_prediction(model).sav'
#The model
pickle.dump(rfc_final,open(filename1,'wb'))
#The count_vectorizer
pickle.dump(c_vect,open(filename2,'wb'))

Loading the model for testing

In [486]:
load_vect=pickle.load(open(filename2,'rb'))
load_model=pickle.load(open(filename1,'rb'))

In [506]:
x=df
col_list=['Full_review','Review_head']
for i in col_list:
    X=load_vect.transform(x[i])
y=df['Rating']

Now testing the model on the original(imbalanced) dataset.

In [507]:
x['Predicted']=load_model.predict(X)
x

,Rating,Review_head,Full_review,Predicted
0,5,Super!,sup bbb prodact amaz bass effect speak sound ...,5
1,5,Great product,sup sound lacky,5
2,5,Must buy!,nic sound system . superb bass audio . awesom...,5
3,4,Worth the money,nic,4
4,4,Nice product,"nic produc , good sound",4
...,...,...,...,...
86304,4,Really Nice,smal .. good begin,4
86305,5,Highly recommended,sup proficit photo graph,5
86306,3,Just okay,"describ ment alumin body 's . 's ab mat , kin...",3
86307,5,Terrific purchase,best stepil low angl shot ... steady cam us c...,5


In [509]:
#Accuracy score and classification report
print(accuracy_score(x['Rating'],x['Predicted']))
print(classification_report(x['Rating'],x['Predicted']))

0.9343637395868334
              precision    recall  f1-score   support

           1       0.95      0.98      0.96     15032
           2       0.93      0.99      0.96      4015
           3       0.85      0.96      0.91      7105
           4       0.82      0.95      0.88     14799
           5       0.99      0.91      0.95     45358

    accuracy                           0.93     86309
   macro avg       0.91      0.96      0.93     86309
weighted avg       0.94      0.93      0.94     86309



Finally,

saving the extracted(scraped) dataset(df) in csv format.

In [510]:
df.to_csv('ratings_prediction(final_dataset).csv')